In [1]:
import pandas as pd


## DATA PROFILING

In [2]:
#CHECK NULL VALUES
def profile_nulls(df):

    for column in df.columns:
        null_count = df[column].isnull().sum()

        if null_count > 0:
            print(f'Column {column} has {null_count} null values.')
        else:
            print(f'Column {column} has no null values.')

### Open files

In [3]:
certificates = pd.read_csv('/home/armando/git/insurance_case/datalake/raw/certificate_dummy.csv')
certificates['fecha_nacimiento'] = pd.to_datetime(certificates['fecha_nacimiento'], format='%Y-%m-%d')
print(certificates.shape)
display(certificates.head(3))

(100, 8)


,id,nombre,email,edad,ciudad,fecha_nacimiento,numero_certificado,sexo
0,1,Luisa Carrera Barroso,eutropiobravo@example.com,74,Zaragoza,1971-03-09,547829,M
1,2,Cristian Murcia,cesar23@example.net,55,Álava,1943-01-08,995112,M
2,3,Gala del Ortega,felipa93@example.com,63,Alicante,1947-01-29,138226,M


In [12]:
terms = pd.read_csv('/home/armando/git/insurance_case/datalake/raw/terms_dummy.csv')
terms['fecha_inicio_vigencia'] = pd.to_datetime(terms['fecha_inicio_vigencia'], format='%Y-%m-%d')
terms['fecha_fin_periodo'] = pd.to_datetime(terms['fecha_fin_periodo'], format='%Y-%m-%d')
print(terms.shape)
display(terms.head(3))

(257, 4)


,id,certificate_number,fecha_inicio_vigencia,fecha_fin_periodo
0,1,20,2021-01-20,2022-01-20
1,2,25,2021-02-03,2022-02-03
2,3,21,2021-02-16,2022-02-16


In [13]:
consultations = pd.read_csv('/home/armando/git/insurance_case/datalake/preprocessed/consultations_dummy.csv')
consultations['fecha_consulta'] = pd.to_datetime(consultations['fecha_consulta'], format='%Y-%m-%d')
print(consultations.shape)
display(consultations.head(3))

(350, 11)


,id,certificate_number,fecha_consulta,specialty,placed_by,day_note_consultation_observation,day_note_next_consultation_pending,day_note_needs_prescription_or_medical_order,patiend_goal,specialist_goal,pause_consultations
0,1,40,2024-08-21,general_medicine,286,La paciente se encuentra orientada en sus tres...,Mujer de la séptima década de la vida con los ...,none,Iniciar seguimiento médico,"Llenado de HC, plan de seguimiento",no
1,2,87,2025-02-15,geriatrics,176,✅ 78.800 kg en cita presencial de estudio de ...,"1️⃣ Verificar potasio sérico, se solicitó cont...",none,"Disminuir peso y grasa corporal, tener hábitos...",✅ Disminuir el riesgo cardiovascular mediante ...,no
2,3,42,2025-06-14,general_medicine,278,Ficha de identificación:\nNombre y cómo le gus...,- Completar esquema de vacunación\n- Realizar ...,none,Prevención primaria,Prevención primaria y secundaria. Validar indi...,no


In [14]:
diagnoses = pd.read_csv('/home/armando/git/insurance_case/datalake/preprocessed/consultation_diagnoses_dummy.csv')
print(diagnoses.shape)
display(diagnoses.head(3))

(432, 2)


,consultation_id,diagnosis
0,1,4917
1,2,3229
2,3,11797


In [15]:
pathologies = pd.read_csv('/home/armando/git/insurance_case/datalake/raw/pathologies.csv')
print(pathologies.shape)
display(pathologies.head(3))

(12423, 3)


,code,id,name
0,A000,1,"COLERA DEBIDO A VIBRIO CHOLERAE O1, BIOTIPO CH..."
1,A001,2,"COLERA DEBIDO A VIBRIO CHOLERAE O1, BIOTIPO EL..."
2,A009,3,COLERA NO ESPECIFICADO


In [16]:
claims = pd.read_csv('/home/armando/git/insurance_case/datalake/raw/claims_dummy.csv')

#CONVERT FROM SPAIN NUMERIC FORMAT TO STANDARD FORMAT
for column in ['OCURRIDO', 'PAGOS', 'COASEGURO', 'IVAREC', 'DEDUCIBLE']:
    claims[column] = claims[column].astype(str)
    claims[column] = claims[column].str.replace('.', '', regex=False)
    claims[column] = claims[column].str.replace(',', '.', regex=False)
    claims[column] = pd.to_numeric(claims[column], errors='raise')

claims.loc[claims['FECHA_PAGO'] == '45491', 'FECHA_PAGO'] = None
claims['FECHA_OCURRIDO'] = pd.to_datetime(claims['FECHA_OCURRIDO'], format='%d/%m/%Y')
claims['FECHA_PAGO'] = pd.to_datetime(claims['FECHA_PAGO'], format='%d/%m/%Y')
claims['FECHA_PRIMERGASTO'] = pd.to_datetime(claims['FECHA_PRIMERGASTO'], format='%d/%m/%Y')
claims['MES_CONT'] = pd.to_datetime(claims['MES_CONT'], format='%d/%m/%Y')

print(claims.shape)
display(claims.head(3))

(51, 18)


,SINIESTRO,STATE,CIE10,DIAGNOSIS,FECHA_OCURRIDO,FECHA_PAGO,FECHA_PRIMERGASTO,OCURRIDO,PAGOS,COASEGURO,IVAREC,DEDUCIBLE,CAUSA,CVE_MES,MES_CONT,TIPO_PAGO,CLASF_PROV,NumCertificado
0,UcV-55067,Huesca,H33,DESPRENDIMIENTO Y DESGARRO DE LA RETINA,2023-09-22,2023-10-06,2023-09-22,43452,80176.71,10862.92,6952.28,25862.07,ENFERMEDAD,202310,2023-10-31,PAGO DIRECTO,Clínica Nuestra Señora,675725
1,Vvh-13843,Cuenca,D04.8,CARCINOMA IN SITU DE LA PIEL DE OTROS SITIOS E...,2023-12-20,2024-01-18,2023-12-20,-16568,23432.00,0.00,3749.11,0.00,ENFERMEDAD,202401,2024-01-31,PAGO DIRECTO,Hospital Metropolitano,577157
2,gZt-84016,Ciudad,L03.9,CELULITIS DE SITIO NO ESPECIFICADO,2024-01-28,2024-02-12,2024-01-28,8885,0.00,0.00,0.00,0.00,ENFERMEDAD,202402,2024-02-29,PAGO DIRECTO,Clínica San Jorge,704444


### Explore Certificates

In [17]:
#CHECK UNICITY OF IDS
shape_01 = certificates['id'].nunique()
shape_02 = certificates.shape[0]
print('Total of certificates:', shape_02)
print('Unique certificate numbers:', certificates['numero_certificado'].nunique())
print(f'Certificate ID unicity check: {shape_01 == shape_02}')

#CHECK AGE DISTRIBUTION
print('Age distribution:')
display(certificates['edad'].describe())

#CHECK CITY VALUES
print('City value counts:')
print(certificates['ciudad'].nunique())
display(certificates['ciudad'].value_counts().head(10))

#CHECK BIRTHDATE VALUES
print('Check birthdate range:')
print(certificates['fecha_nacimiento'].min())
print(certificates['fecha_nacimiento'].max())

#CHECK SEX VALUES
print('Gender value counts:')
display(certificates['sexo'].value_counts())

#CHECK NULLS
profile_nulls(certificates)

display(certificates.head(3))

Total of certificates: 100
Unique certificate numbers: 100
Certificate ID unicity check: True
Age distribution:


count    100.000000
mean      61.840000
std       13.395853
min       40.000000
25%       49.000000
50%       63.500000
75%       72.000000
max       85.000000
Name: edad, dtype: float64

City value counts:
47


ciudad
Salamanca     6
Zaragoza      5
Badajoz       5
Cuenca        5
Segovia       4
Cádiz         4
Valladolid    4
Jaén          4
Lleida        3
Ourense       3
Name: count, dtype: int64

Check birthdate range:
1941-04-26 00:00:00
1985-07-04 00:00:00
Gender value counts:


sexo
M    54
F    46
Name: count, dtype: int64

Column id has no null values.
Column nombre has no null values.
Column email has no null values.
Column edad has no null values.
Column ciudad has no null values.
Column fecha_nacimiento has no null values.
Column numero_certificado has no null values.
Column sexo has no null values.


,id,nombre,email,edad,ciudad,fecha_nacimiento,numero_certificado,sexo
0,1,Luisa Carrera Barroso,eutropiobravo@example.com,74,Zaragoza,1971-03-09,547829,M
1,2,Cristian Murcia,cesar23@example.net,55,Álava,1943-01-08,995112,M
2,3,Gala del Ortega,felipa93@example.com,63,Alicante,1947-01-29,138226,M


### Explore Terms

In [19]:
#CHECK UNICITY OF IDS
shape_01 = terms['id'].nunique()
shape_02 = terms.shape[0]
print('Total of terms:', shape_02)
print(f'Terms ID unicity check: {shape_01 == shape_02}')

#CHECK DATE VALUES
print('Check start date range:')
print(terms['fecha_inicio_vigencia'].min())
print(terms['fecha_inicio_vigencia'].max())

print('Check end date range:')
print(terms['fecha_fin_periodo'].min())
print(terms['fecha_fin_periodo'].max())

subset = terms.loc[terms['fecha_fin_periodo'] < terms['fecha_inicio_vigencia']]
print('Terms with end date before start date:')
print(subset.shape)

#CHECK CERTIFICATES IDS
print('Terms without matching certificate IDs:')
subset = terms.loc[~terms['certificate_number'].isin(certificates['id'])]
print(subset.shape)

print('Certificates without matching terms IDs:')
subset = certificates.loc[~certificates['id'].isin(terms['certificate_number'])]
print(subset.shape)

#CHECK NULLS
profile_nulls(terms)

display(terms.head(3))


Total of terms: 257
Terms ID unicity check: True
Check start date range:
2021-01-20 00:00:00
2024-12-29 00:00:00
Check end date range:
2022-01-20 00:00:00
2025-12-29 00:00:00
Terms with end date before start date:
(0, 4)
Terms without matching certificate IDs:
(0, 4)
Certificates without matching terms IDs:
(0, 8)
Column id has no null values.
Column certificate_number has no null values.
Column fecha_inicio_vigencia has no null values.
Column fecha_fin_periodo has no null values.


,id,certificate_number,fecha_inicio_vigencia,fecha_fin_periodo
0,1,20,2021-01-20,2022-01-20
1,2,25,2021-02-03,2022-02-03
2,3,21,2021-02-16,2022-02-16


### Explore consultations

In [20]:
#CHECK UNICITY OF IDS
shape_01 = consultations['id'].nunique()
shape_02 = consultations.shape[0]
print('Total of rows:', shape_02)
print(f'Consultations ID unicity check: {shape_01 == shape_02}')

#CHECK CERTIFICATES IDS
print('Consultations without matching certificate IDs:')
subset = consultations.loc[~consultations['certificate_number'].isin(certificates['id'])]
print(subset.shape)

print('Certificates without matching consultations IDs:')
subset = certificates.loc[~certificates['id'].isin(consultations['certificate_number'])]
print(subset.shape)

#CHECK CONSULTATION DATE
print('Check consultation date range:')
print(consultations['fecha_consulta'].min())
print(consultations['fecha_consulta'].max())

#CHECK SPECIALTY VALUES
print('Specialty value counts:')
display(consultations['specialty'].value_counts().head(30))

#CHECK DAY NOTES NEEDS PRESCRIPTION
print('Day notes needing prescription value counts:')
display(consultations['day_note_needs_prescription_or_medical_order'].value_counts())

#CHECK PAUSE CONSULTATION
print('Pause consultation value counts:')
display(consultations['pause_consultations'].value_counts())

#CHECK NULLS
profile_nulls(consultations)

display(consultations.head(3))


Total of rows: 350
Consultations ID unicity check: True
Consultations without matching certificate IDs:
(0, 11)
Certificates without matching consultations IDs:
(15, 8)
Check consultation date range:
2021-02-13 00:00:00
2025-11-23 00:00:00
Specialty value counts:


specialty
general_medicine    199
geriatrics           72
gerontology          54
nutrition            25
Name: count, dtype: int64

Day notes needing prescription value counts:


day_note_needs_prescription_or_medical_order
none                   256
medicalOrder            38
medicalPrescription     30
both                    23
Name: count, dtype: int64

Pause consultation value counts:


pause_consultations
no     335
yes      9
Name: count, dtype: int64

Column id has no null values.
Column certificate_number has no null values.
Column fecha_consulta has no null values.
Column specialty has no null values.
Column placed_by has no null values.
Column day_note_consultation_observation has 125 null values.
Column day_note_next_consultation_pending has 35 null values.
Column day_note_needs_prescription_or_medical_order has 3 null values.
Column patiend_goal has 18 null values.
Column specialist_goal has 20 null values.
Column pause_consultations has 6 null values.


,id,certificate_number,fecha_consulta,specialty,placed_by,day_note_consultation_observation,day_note_next_consultation_pending,day_note_needs_prescription_or_medical_order,patiend_goal,specialist_goal,pause_consultations
0,1,40,2024-08-21,general_medicine,286,La paciente se encuentra orientada en sus tres...,Mujer de la séptima década de la vida con los ...,none,Iniciar seguimiento médico,"Llenado de HC, plan de seguimiento",no
1,2,87,2025-02-15,geriatrics,176,✅ 78.800 kg en cita presencial de estudio de ...,"1️⃣ Verificar potasio sérico, se solicitó cont...",none,"Disminuir peso y grasa corporal, tener hábitos...",✅ Disminuir el riesgo cardiovascular mediante ...,no
2,3,42,2025-06-14,general_medicine,278,Ficha de identificación:\nNombre y cómo le gus...,- Completar esquema de vacunación\n- Realizar ...,none,Prevención primaria,Prevención primaria y secundaria. Validar indi...,no


In [53]:
#CHECK DISGNOSIS IDS
print('Total of diagnosis rows:', diagnoses.shape[0])
print('Total of unique consultation IDs in diagnosis:', diagnoses['consultation_id'].nunique())

print('Diagnosis without matching consultations IDs:')
subset = diagnoses.loc[~diagnoses['consultation_id'].isin(consultations['id'])]
print(subset.shape)

print('Consultations without matching diagnosis IDs:')
subset = consultations.loc[~consultations['id'].isin(diagnoses['consultation_id'])]
print(subset.shape)

#CHECK PATHOLOGY IDS
print('Diagnosis without matching pathology IDs:')
subset = diagnoses.loc[~diagnoses['diagnosis'].isin(pathologies['id'])]
print(subset.shape)

#CHECK NULLS
profile_nulls(diagnoses)

display(diagnoses.head(3))

Total of diagnosis rows: 432
Total of unique consultation IDs in diagnosis: 350
Diagnosis without matching consultations IDs:
(0, 2)
Consultations without matching diagnosis IDs:
(0, 11)
Diagnosis without matching pathology IDs:
(0, 2)
Column consultation_id has no null values.
Column diagnosis has no null values.


,consultation_id,diagnosis
0,1,4917
1,2,3229
2,3,11797


### Explore pathologies

In [22]:
#CHECK PATHOLOGY IDS
shape_01 = pathologies['id'].nunique()
shape_02 = pathologies.shape[0]
print('Total of pathologies:', shape_02)
print(f'Pathology ID unicity check: {shape_01 == shape_02}')

#CHECK CODE VALUES
print('Pathology code unique values:')
display(pathologies['code'].nunique())

#CHECK NAME VALUES
print('Pathology name unique values:')
display(pathologies['name'].nunique())

print('Pathology name value counts:')
display(pathologies['name'].value_counts().head(10))

#CHECK NULL VALUES
profile_nulls(pathologies)


display(pathologies.head(3))


Total of pathologies: 12423
Pathology ID unicity check: True
Pathology code unique values:


12423

Pathology name unique values:


12420

Pathology name value counts:


name
OTRAS MALFORMACIONES CONGENITAS DE LA LARINGE                                                                            2
SIFILIS CARDIOVASCULAR                                                                                                   2
ASFIXIA                                                                                                                  2
EFECTO TOXICO DEL CONTACTO CON ANIMALES VENENOSOS: EFECTO TOXICO DEL CONTACTO CON OTROS ANIMALES VENENOSOS               1
EFECTO TOXICO DEL CONTACTO CON ANIMALES VENENOSOS: EFECTO TOXICO DEL CONTACTO CON ANIMAL VENENOSO NO ESPECIFICADO        1
EFECTO TOXICO DE AFLATOXINA Y OTRAS MICOTOXINAS CONTAMINANTES DE ALIMENTOS                                               1
EFECTO TOXICO DE OTRAS SUSTANCIAS Y LAS NO ESPECIFICADAS: CIANURO                                                        1
EFECTO TOXICO DE OTRAS SUSTANCIAS Y LAS NO ESPECIFICADAS: ESTRICNINA Y SUS SALES                                         1
EFECTO TOXI

Column code has no null values.
Column id has no null values.
Column name has no null values.


,code,id,name
0,A000,1,"COLERA DEBIDO A VIBRIO CHOLERAE O1, BIOTIPO CH..."
1,A001,2,"COLERA DEBIDO A VIBRIO CHOLERAE O1, BIOTIPO EL..."
2,A009,3,COLERA NO ESPECIFICADO


### Explore Claims

In [51]:
# CHECK CLAIM IDS
shape_01 = claims['SINIESTRO'].nunique()
shape_02 = claims.shape[0]
print('Total of claims:', shape_02)
print(f'Claim ID unicity check: {shape_01 == shape_02}')

#CHECK STATE VALUES
print('Claim state value counts:')
display(claims['STATE'].nunique())
display(claims['STATE'].value_counts().head(10))

#CHECK CIE VALUES
print('CIE code unique values:')
print(claims['CIE10'].nunique())
print('CIE diagnosis unique:')
print(claims['DIAGNOSIS'].nunique())

#CHACK DATE VALUES
print('Check occurrence date range:')
print(claims['FECHA_OCURRIDO'].min())
print(claims['FECHA_OCURRIDO'].max())

print('Check payment date range:')
print(claims['FECHA_PAGO'].min())
print(claims['FECHA_PAGO'].max())

print('Check first expense date range:')
print(claims['FECHA_PRIMERGASTO'].min())
print(claims['FECHA_PRIMERGASTO'].max())

print('Check MES_CONT date range:')
print(claims['MES_CONT'].min())
print(claims['MES_CONT'].max())

print('Check dates consistency:')
subset = claims.loc[claims['FECHA_PRIMERGASTO'] < claims['FECHA_OCURRIDO']]
print('Claims with first expense date before occurrence date:', subset.shape[0])

subset = claims.loc[(claims['FECHA_PAGO'].notnull()) & (claims['FECHA_PAGO'] < claims['FECHA_OCURRIDO'])]
print('Claims with payment date before occurrence date:', subset.shape[0])

subset = claims.loc[(claims['FECHA_PAGO'].notnull()) & (claims['FECHA_PAGO'] < claims['FECHA_PRIMERGASTO'])]
print('Claims with payment date before first expense date:', subset.shape[0])

#CHECK FLOAT COLUMNS STATISTICS
print('CHECK AMOUNT STATISTICS')
display(claims[['OCURRIDO', 'PAGOS', 'COASEGURO', 'IVAREC', 'DEDUCIBLE']].describe())

#CAUSES
print('Cause value counts:')
display(claims['CAUSA'].value_counts().head(10))

#RELATION MONTHS
print('Check relation between CVE_MES and MES_CONT:')
display(claims[['CVE_MES', 'MES_CONT']].drop_duplicates().head(30))

#PAYMENT TYPE VALUES
print('Payment type value counts:')
display(claims['TIPO_PAGO'].value_counts())

#PROVIDERS
print('Provider value counts:')
display(claims['CLASF_PROV'].value_counts().head(10))

#RELATION BETWEEN CLAIMS AND CERTIFICATES
print('Claims without matching certificate IDs:')
subset = claims.loc[~claims['NumCertificado'].isin(certificates['numero_certificado'])]
print(subset.shape)

print('Certificates without matching claims IDs:')
subset = certificates.loc[~certificates['numero_certificado'].isin(claims['NumCertificado'])]
print(subset.shape)

#CHECK NULLS
profile_nulls(claims)

Total of claims: 51
Claim ID unicity check: True
Claim state value counts:


32

STATE
Baleares      4
Valladolid    3
Granada       3
Vizcaya       3
Asturias      2
Badajoz       2
Segovia       2
Cuenca        2
Tarragona     2
La Rioja      2
Name: count, dtype: int64

CIE code unique values:
46
CIE diagnosis unique:
46
Check occurrence date range:
2023-09-22 00:00:00
2025-08-22 00:00:00
Check payment date range:
2023-10-06 00:00:00
2025-09-18 00:00:00
Check first expense date range:
2023-09-22 00:00:00
2025-08-22 00:00:00
Check MES_CONT date range:
2023-10-31 00:00:00
2025-09-30 00:00:00
Check dates consistency:
Claims with first expense date before occurrence date: 0
Claims with payment date before occurrence date: 0
Claims with payment date before first expense date: 0
CHECK AMOUNT STATISTICS


,OCURRIDO,PAGOS,COASEGURO,IVAREC,DEDUCIBLE
count,51.000000,51.000000,51.000000,51.000000,51.000000
mean,44353.666667,67723.563529,974.514902,8764.243529,3652.653529
std,92520.178522,96677.552255,3091.949345,15862.853912,9152.608026
min,-84845.000000,0.000000,0.000000,0.000000,0.000000
25%,2434.000000,13134.565000,0.000000,0.000000,0.000000
50%,20957.000000,31235.530000,0.000000,1821.580000,0.000000
75%,49552.000000,94688.720000,0.000000,14355.915000,0.000000
max,575813.000000,575812.990000,17241.380000,92130.080000,29156.140000


...

Payment type value counts:


TIPO_PAGO
PAGO DIRECTO    51
Name: count, dtype: int64

Provider value counts:


CLASF_PROV
Clínica Nuestra Señora     8
Clínica San Jorge          8
Hospital Regional          5
Clínica del Valle          5
Hospital del Mar           4
Instituto Médico           4
Hospital General           4
Instituto de Salud         3
Hospital Metropolitano     3
Clínica Privada Central    3
Name: count, dtype: int64

Claims without matching certificate IDs:
(0, 18)
Certificates without matching claims IDs:
(56, 8)
Column SINIESTRO has no null values.
Column STATE has no null values.
Column CIE10 has no null values.
Column DIAGNOSIS has no null values.
Column FECHA_OCURRIDO has no null values.
Column FECHA_PAGO has 1 null values.
Column FECHA_PRIMERGASTO has no null values.
Column OCURRIDO has no null values.
Column PAGOS has no null values.
Column COASEGURO has no null values.
Column IVAREC has no null values.
Column DEDUCIBLE has no null values.
Column CAUSA has no null values.
Column CVE_MES has no null values.
Column MES_CONT has no null values.
Column TIPO_PAGO has no null values.
Column CLASF_PROV has no null values.
Column NumCertificado has no null values.


In [82]:
display(claims.head(3))

,SINIESTRO,STATE,CIE10,DIAGNOSIS,FECHA_OCURRIDO,FECHA_PAGO,FECHA_PRIMERGASTO,OCURRIDO,PAGOS,COASEGURO,IVAREC,DEDUCIBLE,CAUSA,CVE_MES,MES_CONT,TIPO_PAGO,CLASF_PROV,NumCertificado
0,UcV-55067,Huesca,H33,DESPRENDIMIENTO Y DESGARRO DE LA RETINA,22/09/2023,06/10/2023,22/09/2023,43.452,"80176,71","10862,92","6952,28","25862,07",ENFERMEDAD,202310,31/10/2023,PAGO DIRECTO,Clínica Nuestra Señora,675725
1,Vvh-13843,Cuenca,D04.8,CARCINOMA IN SITU DE LA PIEL DE OTROS SITIOS E...,20/12/2023,18/01/2024,20/12/2023,-16.568,23432,0,"3749,11",0,ENFERMEDAD,202401,31/01/2024,PAGO DIRECTO,Hospital Metropolitano,577157
2,gZt-84016,Ciudad,L03.9,CELULITIS DE SITIO NO ESPECIFICADO,28/01/2024,12/02/2024,28/01/2024,8.885,0,0,0,0,ENFERMEDAD,202402,29/02/2024,PAGO DIRECTO,Clínica San Jorge,704444
